In [28]:
import gurobipy as gp
from gurobipy import GRB

In [29]:
# === Sets ===
A_s = [0, 1, 2, 3]                                                # Locations (0 is hub)
T = [3]                                                              # Days
V = ['car', 'bike', 'walk']                        # Vehicle types
H = list(range(8, 20))                                                  # Hours of the day  

# === Parameters ===
# Daily time limits by vehicle per hour
Q_h = {
    h: {
        'car': 3,
        'bike': 5,
        'e_bike': 5,
        'scooter': 4,
        'walk': 6
    } for h in H
}

cost_rate_v = {
    'car': 10,
    'bike': 5,
    'e_bike': 3,
    'scooter': 4,
    'walk': 0
}

# day based costs, people based costs

# Travel times per vehicle
vehicle_travel_time = {
    'car': 1,
    'bike': 2,
    'e_bike': 1.5,
    'scooter': 1.8,
    'walk': 4
}

t_ij_s_v = {}

for v, time_per_unit in vehicle_travel_time.items():
    for i in A_s:
        for j in A_s:
            if i != j:
                t_ij_s_v[(i, j, v)] = time_per_unit

# Visit times per vehicle
tv_j_s_v = {(j, v): 1 for j in A_s for v in V}
for v in V:
    tv_j_s_v[(0, v)] = 0  # depot has no service time

# Availability for a location at a given time for a given vehicle
open_j_h_v = {}

poi_open_hours = {
    1: (9, 17),
    2: (9, 17),
    3: (9, 17),
    4: (10, 16),
    5: (11, 15)
}

for j in A_s:
    for h in H:
        for v in V:
            if j == 0:
                open_j_h_v[(j, h, v)] = 1  # depot always open
            elif j in poi_open_hours:
                start, end = poi_open_hours[j]
                open_j_h_v[(j, h, v)] = 1 if start <= h < end else 0
            else:
                open_j_h_v[(j, h, v)] = 0  # default closed

In [30]:
# === Compute q_j for each j: average t_{i→j} over all i plus service time at j ===
q_j = {}
for j in A_s:
    # collect all travel times into j (across all vehicles if multimodal; here single 'car')
    incoming = [
        t_ij_s_v[(i, j, v)]
        for i in A_s for v in V
        if i != j and (i, j, v) in t_ij_s_v
    ]
    if incoming:
        avg_travel = sum(incoming) / len(incoming)
        # service time at j: pick any vehicle (service is vehicle-specific)
        # here: tv_j_s_v[(j, v)] is the same for all v
        svc = tv_j_s_v[(j, V[0])]
        q_j[j] = avg_travel + svc
    else:
        q_j[j] = 0.0   # depot or isolated node

In [31]:
# === Model ===
model = gp.Model("MultimodalVehicleRouting")

# === Variables ===
x = model.addVars(A_s, A_s, T, H, V, vtype=GRB.BINARY, name="x")
X = model.addVars(T, vtype=GRB.BINARY, name="X")  # Trips from depot per vehicle
u = model.addVars(A_s, H, V, vtype=GRB.CONTINUOUS, name="u")
z = model.addVars(T, H, V, vtype=GRB.BINARY, name="z")  # is vehicle currently active

# === Objective ===
lambda_cost = 0.001  # smaller penalty

visit_reward = 10    # scale up visits to balance cost scale

model.setObjective(
    gp.quicksum(
        x[i, j, t, h, v] * visit_reward
        for i in A_s for j in A_s
        for t in T for h in H for v in V
        if i != j and (i, j, v) in t_ij_s_v
    )
    - lambda_cost * gp.quicksum(
        x[i, j, t, h, v] * t_ij_s_v[i, j, v] * cost_rate_v[v]
        for i in A_s for j in A_s
        for t in T for h in H for v in V
        if i != j and (i, j, v) in t_ij_s_v
    ),
    GRB.MAXIMIZE
)

""" future priority system objective
priority = {1: 5, 2: 3, 3: 4}
model.setObjective(
    gp.quicksum(x[i, j, t, h, v] * priority.get(j, 1)
                for i in A_s for j in A_s for t in T for h in H for v in V
                if i != j and (i, j, v) in t_ij_s_v),
    GRB.MAXIMIZE
)
"""

' future priority system objective\npriority = {1: 5, 2: 3, 3: 4}\nmodel.setObjective(\n    gp.quicksum(x[i, j, t, h, v] * priority.get(j, 1)\n                for i in A_s for j in A_s for t in T for h in H for v in V\n                if i != j and (i, j, v) in t_ij_s_v),\n    GRB.MAXIMIZE\n)\n'

In [ ]:
# === Constraints ===

# No self-loops
model.addConstrs(
    (gp.quicksum(x[i, i, t, h, v] for t in T for h in H for v in V) == 0 for i in A_s),
    name="NoSelfLoops"
)

# Flow conservation
model.addConstrs((
    gp.quicksum(x[i, j, t, h, v] for i in A_s if (i, j, v) in t_ij_s_v) ==
    gp.quicksum(x[j, k, t, h, v] for k in A_s if (j, k, v) in t_ij_s_v)
    for j in A_s for t in T for h in H for v in V), name="FlowConservation"
)

# One visit per node (excluding depot)
model.addConstrs((
    gp.quicksum(x[i, j, t, h, v] 
                for t in T for h in H for v in V for i in A_s 
                if i != j and (i, j, v) in t_ij_s_v) <= 1
    for j in A_s if j != 0), name="OneVisit"
)

# Depot departure matches X[t], X shouldnt be depending on h and v ?
model.addConstrs((
    gp.quicksum(x[0, j, t, h, v] 
                for h in H for v in V for j in A_s if j != 0 and (0, j, v) in t_ij_s_v) == X[t]
    for t in T), name="DepotStarts"
)

# Time budget constraint for each vehicle per day
model.addConstrs((
    gp.quicksum(x[i, j, t, h, v] * (t_ij_s_v[i, j, v] + tv_j_s_v[j, v])
                for i in A_s for j in A_s if i != j and (i, j, v) in t_ij_s_v) <= Q_h[h][v]
    for t in T for h in H for v in V), name="TimeBudget"
)

# Monotonicity: Xt, ≥ Xt+1,
model.addConstrs((
    X[t1] >= X[t2]
    for t1, t2 in zip(T[:-1], T[1:])
), name="MonotonicTrips")

# Availability (based on opening hours)
model.addConstrs((
    x[i, j, t, h, v] <= open_j_h_v[j, h, v]
    for i in A_s for j in A_s for t in T for h in H for v in V if (i, j, v) in t_ij_s_v
), name="Availability"
)

model.addConstrs(
    (
        x[i, j, t, h, v] <= z[t, h, v]
        for i in A_s for j in A_s for t in T for h in H for v in V
        if i != j and (i, j, v) in t_ij_s_v
    ),
    name="ArcImpliesVehicleUse"
)

model.addConstrs(
    (
        gp.quicksum(z[t, h, v] for h in H) <= 3  # or any per-day limit
        for t in T for v in V
    ),
    name="MaxHoursPerVehicle"
)

model.addConstrs((
        gp.quicksum(z[t, h, v] for v in V) <= 1
        for t in T for h in H
    ),
    name="OnlyOneVehiclePerHour"
)

# Sub-tour Elimination Constraints (MTZ-like)
model.addConstrs((
    u[i, h, v] - u[j, h, v] + Q_h[h][v] * (1 - x[i, j, t, h, v]) >= 0
    for i in A_s for j in A_s if i != j 
    for t in T for h in H for v in V if (i, j, v) in t_ij_s_v
), name="MTZ1"
)

model.addConstrs((
    u[j, h, v] >= t_ij_s_v[i, j, v]
    for i in A_s for j in A_s if j != 0 
    for t in T for h in H for v in V if (i, j, v) in t_ij_s_v
), name="MTZ2"
)

model.addConstrs((
    u[i, h, v] <= Q_h[h][v]
    for i in A_s for h in H for v in V
), name="MaxTime"
)

{(0, 8, 'car'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8, 'bike'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8, 'walk'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9, 'car'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9, 'bike'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9, 'walk'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10, 'car'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10, 'bike'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10, 'walk'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11, 'car'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11, 'bike'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11, 'walk'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12, 'car'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12, 'bike'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12, 'walk'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13, 'car'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13, 'bike'): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13, 

In [33]:
# === Solve ===
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1967 rows, 757 columns and 5005 nonzeros
Model fingerprint: 0xa6a02b14
Variable types: 144 continuous, 613 integer (613 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 1967 rows and 757 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 19.98 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.998000000000e+01, best bound 1.998000000000e+01, gap 0.0000%


In [34]:
# === Output All Trips ===
if model.status == GRB.OPTIMAL:
    print(f"\n✅ Optimal Objective Value: {model.objVal:.2f}\n")
    for t in T:
        for h in H:
            for v in V:
                # collect selected arcs
                arcs = [(i, j) for i in A_s for j in A_s
                        if i != j and (i, j, v) in t_ij_s_v and x[i, j, t, h, v].X > 0.5]

                if not arcs:
                    continue

                print(f"Day {t} @ {h:02d}:00, with Vehicle {v}:")

                # build mapping of successors and predecessors
                succ = {i: j for i, j in arcs}
                pred = {j: i for i, j in arcs}

                # find all starting points (0 → j arcs)
                starts = [j for i, j in arcs if i == 0]

                # reconstruct all routes starting from depot
                used = set()
                for start in starts:
                    if start in used:
                        continue

                    route = [0, start]
                    curr = start
                    used.add(0)
                    used.add(start)

                    while curr in succ and succ[curr] != 0 and succ[curr] not in used:
                        curr = succ[curr]
                        route.append(curr)
                        used.add(curr)

                    # end at depot if possible
                    if curr in succ and succ[curr] == 0:
                        route.append(0)
                        used.add(0)

                    route_str = " → ".join(str(n) for n in route)
                    print(f"  {route_str}")
else:
    print("❌ No optimal solution found.")

## only end at home at the end of the day


✅ Optimal Objective Value: 19.98

Day 3 @ 09:00, with Vehicle car:
  0 → 1 → 0
